<center><h1 style="font-size:2em;color:#2467C0"> Datos del INEGI - Proyecto  </h1></center>
<br>
<table>
<col width="550">
<col width="450">
<tr>
<td><img src="figuras/inegi.png" align="middle" style="width:500"/></td>
<td>
    
El **INEGI** es un organismo público autónomo responsable de normar y coordinar el Sistema Nacional de Información Estadística y Geográfica, así como de captar y difundir información de México en cuanto al territorio, los recursos, la población y economía, que permita dar a conocer las características de nuestro país y ayudar a la toma de decisiones.
    
</td>
</tr>
</table>

# API del Banco de indicadores

La API de Indicadores del INEGI permite consultar los datos y metadatos de los indicadores disponibles a nivel nacional, por entidad federativa y municipio. Puede utilizar la API para crear aplicaciones que muestren la información directamente de las bases de datos del INEGI en el preciso momento en que se actualiza.
[[1]](https://www.inegi.org.mx/servicios/api_indicadores.html) 

<img src="figuras/Inegi_token.jpg" align="middle" style="width:1000px"/>


## Consultas INEGI

A continuación se mostrarán cuatro gráficas y un mapa con información obtenida del INEGI con el método mencionado anteriormente.

Importamos las librerías

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import numpy as np
import pandas as pd
import requests
import json

### Analfabetismo en México

El URL siguiente corresponde a la información del *porcentaje total de analfabetas en México de forma histórica*.

In [ ]:
url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/3108001001/es/0700/false/BISE/2.0/2df7d990-4bbc-c1bd-f33e-2da2413413df?type=json'
respuesta = requests.get(url)
if respuesta.status_code == 200:
    contenido = json.loads(respuesta.content)

contenido

Para acceder a la información se debe ejecutar éste código repetidas veces.

In [ ]:
contenido['Series'][0]['OBSERVATIONS']

El código para acceder de forma directa al conjunto de datos es el siguiente:

In [ ]:
url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/3108001001/es/0700/false/BISE/2.0/2df7d990-4bbc-c1bd-f33e-2da2413413df?type=json'
respuesta = requests.get(url)
if respuesta.status_code == 200:
    contenido = json.loads(respuesta.content)
    # Cargar el contenido a la variable 'datos'
    datos = contenido['Series'][0]['OBSERVATIONS']
    pob_analfabeta = []
    periodo = []
    for dato in datos:
        pob_analfabeta.append(round(float(dato['OBS_VALUE']),2))
        periodo.append(str(dato['TIME_PERIOD']))
    
print('Lista del porcentaje de la población analfabeta en México: \n', pob_analfabeta)
print('Periodo: \n', periodo)


Con los datos anteriores se puede crear un gráfico.

In [ ]:
# Gráfico con estilo MATLAB
plt.figure(figsize=(10,4))
plt.title('Porcentaje de analfabetismo en México')
plt.bar(periodo, pob_analfabeta, facecolor = 'yellowgreen', width = 0.6)
plt.xlabel('Periodo')
plt.ylabel('Porcentaje de la población')
plt.show()

El siguiente gráfico consiste en el porcentaje de población analfabeta dividida por sexo.

In [ ]:
# Porcentaje de analfabetas mujeres
url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/3108001003/es/0700/false/BISE/2.0/2df7d990-4bbc-c1bd-f33e-2da2413413df?type=json'
respuesta = requests.get(url)
if respuesta.status_code == 200:
    contenido = json.loads(respuesta.content)
    # Cargar el contenido a la variable 'datos'
    datos = contenido['Series'][0]['OBSERVATIONS']
    pob_analfabeta_muj = []
    periodo = []
    for dato in datos:
        pob_analfabeta_muj.append(round(float(dato['OBS_VALUE']),2))
        periodo.append(str(dato['TIME_PERIOD']))
    
print('Lista del Porcentaje de analfabetas mujeres en México: \n', pob_analfabeta_muj)
print('Periodo: \n', periodo)

In [ ]:
# Porcentaje de analfabetas hombres
url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/3108001002/es/0700/false/BISE/2.0/2df7d990-4bbc-c1bd-f33e-2da2413413df?type=json'
respuesta = requests.get(url)
if respuesta.status_code == 200:
    contenido = json.loads(respuesta.content)
    # Cargar el contenido a la variable 'datos'
    datos = contenido['Series'][0]['OBSERVATIONS']
    pob_analfabeta_hom = []
    periodo = []
    for dato in datos:
        pob_analfabeta_hom.append(round(float(dato['OBS_VALUE']),2))
        periodo.append(str(dato['TIME_PERIOD']))
    
print('Lista del Porcentaje de analfabetas hombres en México: \n', pob_analfabeta_hom)
print('Periodo: \n', periodo)

In [ ]:
# DataFrame del analfabetismo en Méxicos
df_analfabetismo = pd.DataFrame()
df_analfabetismo['periodo'] = periodo
df_analfabetismo['porcentaje_mujeres'] = pob_analfabeta_muj
df_analfabetismo['porcentaje_hombres'] = pob_analfabeta_hom

df_analfabetismo

In [ ]:
 # Definiendo posiciones auxiliares de los ejes
x_vals = np.arange(len(periodo[:]))
ancho = 0.25

 # Estilo MATLAB
plt.figure(figsize=(10,4))

plt.bar(x_vals - ancho/2, df_analfabetismo['porcentaje_mujeres'], width= ancho, label='Mujeres')
plt.bar(x_vals + ancho/2, df_analfabetismo['porcentaje_hombres'], width=ancho, label='Hombres')

plt.xticks(x_vals, periodo)
plt.title('Analfabetismo en México')
plt.xlabel('Periodo')
plt.ylabel('Porcentaje de la población')
plt.legend()

plt.show()

### Porcentaje de la población analfabeta de 15 años y más por estado

Cargamos el DataFrame con la clave de la entidad federativa y su abreviación

In [ ]:
df_estados = pd.read_csv('datos/Catalogo_entidades.csv')[:32]
df_estados.head(8)

In [ ]:
lista_porcentajes = []
#3108001001
for numero in df_estados['CLAVE_ENTIDAD']:
    if numero < 10:
        idnum = '0700000'+str(numero)
    else:
        idnum = '070000'+str(numero)
    #3108001001    
    #url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/3108001001/es/' + idnum + '/true/BISE/2.0/2df7d990-4bbc-c1bd-f33e-2da2413413df?type=json'
    #1002000041
    url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000041/es/' + idnum + '/true/BISE/2.0/2df7d990-4bbc-c1bd-f33e-2da2413413df?type=json'
    response = requests.get(url)
    if response.status_code == 200:
        content = json.loads(response.content)
        lista_porcentajes.append(round(100 - float(content['Series'][0]['OBSERVATIONS'][0]['OBS_VALUE']),4))
    
df_estados['PORCENTAJES'] = lista_porcentajes
df_estados.head(30)

In [ ]:
# Gráfica estilo seaborn
fig, ax = plt.subplots(figsize=(5,8))
sns.barplot(data = df_estados, y = 'ENTIDAD_FEDERATIVA', x = 'PORCENTAJES', color = 'lightcoral', order = df_estados.sort_values('PORCENTAJES', ascending = False)['ENTIDAD_FEDERATIVA'])

In [ ]:
estados_geo = ('datos/mexico_estados_geo.json')
map = folium.Map(location=[24,-102], zoom_start=5,min_zoom=5,max_zoom=6)

titulo="Analfabetismo en México"

folium.Choropleth(geo_data=estados_geo, data=df_estados, 
             columns=['ABREVIATURA', 'PORCENTAJES'],
             key_on='feature.id',
             fill_color='Reds', fill_opacity=0.9, line_opacity=0.9,
             legend_name=titulo).add_to(map)
map

### Analfabetismo en Yucatán

Cargamos el DataFrame de los municipios [[2]](https://ide.sedatu.gob.mx/layers/geonode:a__31_MUN_INEGI_2020/metadata_detail)

In [ ]:
df_municipios = pd.read_csv('datos/Catalogo_municipios.csv')
df_municipios


In [ ]:
df_municipios = pd.read_csv('datos/Catalogo_municipios.csv')#[:106]
df_municipios.tail()

Obtenemos la información del INEGI

In [ ]:
# Por municipio
lista_porcentajes = []
#070000310001
for numero in df_municipios['CVE_MUN']:
    if numero < 10:
        idnum = '07000031000'+ str(numero)
    elif numero < 100 :
        idnum = '0700003100'+ str(numero)
    else:
        idnum = '070000310'+ str(numero)
        
    url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000041/es/' + idnum + '/true/BISE/2.0/2df7d990-4bbc-c1bd-f33e-2da2413413df?type=json'
    response = requests.get(url)
    if response.status_code == 200:
        content = json.loads(response.content)
        lista_porcentajes.append(round(100 - float(content['Series'][0]['OBSERVATIONS'][0]['OBS_VALUE']),4))
    
df_municipios['PORCENTAJES'] = lista_porcentajes
df_municipios.head()


In [ ]:
municipios_geo = ('datos/yucatan_municipios_geo.json')
map = folium.Map(location = [20.50, -88.80], zoom_start = 8, max_zoom=13, min_zoom=7)

titulo="Analfabetismo en Yucatán"

folium.Choropleth(geo_data=municipios_geo, data=df_municipios, 
             columns=['FID', 'PORCENTAJES'],
             key_on='feature.id',
             fill_color='Reds', fill_opacity=0.9, line_opacity=0.9,
             legend_name=titulo).add_to(map)
map

In [ ]:
# Gráfica estilo seaborn
fig, ax = plt.subplots(figsize=(5,25))
sns.barplot(data = df_municipios, y = 'NOMGEO', x = 'PORCENTAJES', color = 'lightcoral', order = df_municipios.sort_values('PORCENTAJES', ascending = False)['NOMGEO'])

### Gastos en educación como porcentaje del PIB

Serie histórica de gastos en educación

In [ ]:
# Gastos en educación, registro histórico
url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/6207067825/es/0700/false/BISE/2.0/2df7d990-4bbc-c1bd-f33e-2da2413413df?type=json'
respuesta = requests.get(url)
if respuesta.status_code == 200:
    contenido = json.loads(respuesta.content)
    # Cargar el contenido a la variable 'datos'
    datos = contenido['Series'][0]['OBSERVATIONS']
    gastos_educacion = []
    periodo = []
    for dato in datos:
        gastos_educacion.append(round(float(dato['OBS_VALUE']),4))
        periodo.append(str(dato['TIME_PERIOD']))
    
print('Lista de gastos realizados en educación: \n', gastos_educacion)
print('Periodo: \n', periodo)

In [ ]:
# Base de datos

df_gasto_educ = pd.DataFrame()
df_gasto_educ['PERIODO'] = periodo
df_gasto_educ['PORCENTAJE'] = gastos_educacion

fig, ax = plt.subplots(figsize = (10,4))
sns.lineplot(data = df_gasto_educ, x = 'PERIODO', y = 'PORCENTAJE', linewidth=2.5, color= 'green')
plt.xticks(rotation = 90)
plt.show()
